In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import mysql.connector
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from pandas.core import datetools
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

sns.set_style('darkgrid')
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None

/home/gspc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/gspc/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/gspc/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/gspc/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/gspc/anaconda3/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/gspc/anaconda3/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

Se

In [ ]:
def create_connection():
    cnx2 = mysql.connector.connect(host='localhost',
                                   user='root', password='MyNewPass',
                                   database='Weather_Data')
    return cnx2

In [ ]:
def select_data():
    conn = create_connection()
    cursor = conn.cursor()
    print('Extracting Data')
    query = "Select * from weather where Observation_date <= '2018-06-01 00:00:00';"
    cursor.execute(query, )
    result = cursor.fetchall()
    print('Extracted Data')
        
    cursor.close()
    conn.close()
    
    return result
    

In [ ]:
def null_values(data):
    print(data.isnull().sum())

In [ ]:
def change_structure(data):
    cols = ['date', 'rain', 'temp', 'wetb', 'dewpt','vappr', 'rhum', 'msl', 'wdsp', 'wddir','height','latitude', 'longitude', 'station','county']
    data = pd.DataFrame(data, columns=cols)
    
    #null_values(data)
    data.fillna(0, inplace=True)
    return data

In [ ]:
def split_time(data):
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year.astype(np.uint16)
    data['month'] = data['date'].dt.month.astype(np.uint8)
    data['day'] = data['date'].dt.day.astype(np.uint8)
    data['hour'] = data['date'].dt.hour.astype(np.uint8)
    
    return data

In [ ]:
def generate_season(data):
    data['season'] = pd.cut(data['month'], bins=[0,1,4,7,10,14], labels=['Winter','Spring','Summer','Autumn','Winter2']).str.replace('Winter2','Winter')
    data['season'] = data['season'].astype('category')
    return data

In [ ]:
def time_features_to_categorical(data):
    #data['year'] = data['year'].astype(str)
    #data['month'] = data['month'].astype(str)
    #data['day'] = data['day'].astype(str)
    #data['hour'] = data['hour'].astype(str)
    
    data['year'] = data['year'].astype('category')
    data['month'] = data['month'].astype('category')
    data['day'] = data['day'].astype('category')
    data['hour'] = data['hour'].astype('category')
    
    return data

In [ ]:
def convert_wdsp(data):
    data['wdsp'] = data['wdsp'] * 1.852
    
    return data

In [ ]:
def update_rhum_values(data):
    print('Updating relative humidity values <= 0')
    rhum_mean = data['rhum'].mean()
    data.loc[data['rhum'] <= 0, 'rhum'] = rhum_mean
    
    return data

In [ ]:
def update_wetb_values(data):
    print('Updating wet bulb air temperature values <= 0')
    wetb_mean = data['wetb'].mean()
    data.loc[data['wetb'] <= -40, 'wetb'] = wetb_mean
    
    return data

In [ ]:
def update_dewpt_values(data):
    print('Updating dew point air temperature values <= 0')
    dewpt_mean = data['dewpt'].mean()
    data.loc[data['dewpt'] <= -20,'dewpt'] = dewpt_mean
    
    return data

In [ ]:
def update_msl_values(data):
    print('Updating mean sea level pressure values <= 0')
    msl_mean = data['msl'].mean() 
    data.loc[data['msl'] < 940, 'msl'] = msl_mean
    
    return data

In [ ]:
def update_vappr_values(data):
    print('Updating vapour pressure values <= 0')
    vappr_mean = data['vappr'].mean()
    data.loc[data['vappr'] <= 0, 'vappr'] = vappr_mean
    
    return data

In [ ]:
def binarize_categories(data):
    print('Binarizing categorical data')
    
    categorical_columns = []

    for col in data.columns:
        if isinstance(data[col][0], str):
            print('Changing ', col, ' to categorical')
            categorical_columns.append(col)
            encoder = LabelBinarizer()
            data[col] = encoder.fit_transform(data[col])
            
    #data = delete_original_categories(categorical_columns, data)        
    return data

In [ ]:
def delete_original_categories(categorical_columns, data):
    print('Deleting original categories')
    for col in categorical_columns:
        if col in data.columns:
            del(data[col])
            gc.collect()
    return data

In [ ]:
def remove_unwanted_features(data):
    print('Removing features')
    data = data.drop( columns = ['date', 'height', 'latitude', 'longitude', 'station', 'county'])
    
    return data

In [ ]:
def time_lagged_features(data, shift_freq, shift_name):
    print('Generating time lagged features')
    
    temp = data
    temp.drop(columns=['season'], inplace=True)
    
    cols = ['year', 'month', 'day', 'hour', 'rain', 'temp', 'wetb', 'dewpt','vappr', 'rhum', 'msl', 'wdsp', 'wddir']
    new_cols = []
    
    for item in cols:
        item = item + shift_name 
        new_cols.append(item)
    
    temp = temp.shift(shift_freq)
    temp.columns = new_cols
    
    temp.reset_index(drop=True, inplace=True)
    data.reset_index(drop=True, inplace=True)
    data = data.join(temp)

    return data

In [ ]:
def sort_data(data):
    data = data.sort_values(['year','month','day','hour'])
    return data

In [ ]:
def aggregate_data(data):
    #may need to drop season feature then recreate it
    #['rain','temp', 'wetb', 'dewpt', 'vappr', 'rhum', 'msl', 'wdsp', 'wddir']
    print('Aggregating Data')
    season = data['season']
    data = data.groupby( by=['year','month','day','hour']).mean().reset_index()
    sorted_data = sort_data(data)
    data['season'] = season
    
    print(len(sorted_data),' observations available for training')
    
    return sorted_data

In [ ]:
def convert_to_supervised(data):
    temp = data
    temp = temp['rain'].shift(-1)
    
    data['target_rainfall'] = temp
    
    data.drop(data.index[len(data)-1], inplace=True)
    data.drop(data.index[0], inplace=True)
    data.fillna(data.mean(), inplace=True)    
    data.reset_index(inplace=True, drop=True)
    
    return data

In [ ]:
def get_target(data):
    target = data['target_rainfall']
    
    return target

In [ ]:
def normalise_data(data):
    print('Normalising test data')
    data.drop(columns = ['target_rainfall'], inplace=True)
    column_names = data.columns
    scaler = MinMaxScaler()
    normalised_data = scaler.fit_transform(data)

    normalised_data = pd.DataFrame(normalised_data, columns=column_names)
    
    return normalised_data

In [ ]:
def get_test(data):
    print('Getting test set')
    
    #train on all years except the last year
    test_data = 20 *365*24
    
    
    train = data[:test_data]
    test = data[test_data:]
    
    Xtrain, Ytrain = train.loc[:, train.columns != 'rain'], train.loc[:, train.columns == 'rain']
    Xtest, Ytest = test.loc[:, test.columns != 'rain'], test.loc[:, test.columns == 'rain']
    
    Xtrain = Xtrain.values # convert to numpy array
    Xtest = Xtest.values
    
    Xtrain = Xtrain.reshape((Xtrain.shape[0], 1, Xtrain.shape[1]))
    Xtest = Xtest.reshape((Xtest.shape[0], 1, Xtest.shape[1]))
    
    print(Xtrain.shape)
    print(Xtest.shape)
    
    # design network
    model = Sequential()
    model.add(LSTM(100, input_shape=(Xtrain.shape[1], Xtrain.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    # fit network
    model.fit(Xtrain, Ytrain, epochs=100, batch_size=1, verbose=2, shuffle=False)
    # plot history
    #plt.plot(history.history['loss'], label='train')
    #plt.plot(history.history['val_loss'], label='test')
    #plt.legend()
    #plt.show()
    
    
   
    #return [train, test]

In [ ]:
data = select_data()
data = change_structure(data)
data = split_time(data)
data = generate_season(data)
#data = time_features_to_categorical(data)
data = remove_unwanted_features(data)
data = convert_wdsp(data)
data = sort_data(data)
data = binarize_categories(data)

data = update_rhum_values(data)
data = update_wetb_values(data)
data = update_dewpt_values(data)
data = update_msl_values(data)
data = update_vappr_values(data)

data.reset_index(drop=True, inplace=True)
data = aggregate_data(data)
data = time_lagged_features(data, 1, '1hr')

data.head()

In [ ]:
data = convert_to_supervised(data)

In [ ]:
target = get_target(data)
data = normalise_data(data)

In [ ]:
get_test(data)
#train = datasets[0]
#test = datasets[1]

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# Important steps

Shift data so its supervised learning problem 

Aggregate data (for future iterations)

Normalise data using standard scaler

### Xgboost 1st Run (Try on cpu and gpu)

Raw features only - remove height, station, county, longitude and latitude

### Xgboost 2nd Run

Shift features to include previous hour. Will first only include the rainfall from the previous hour

### Xgboost 3rd Run

Shift features to include 2 hours previous 

### Xgboost 3rd Run

Shift features to include 1 **and** 2 hours previous 


## Potential Features
minus the monthly, yearly , seasonal average, lagged features, median values?

aggregate data, difference between current and previous values

num splits equal to num distinct years by 12 (months)

31 * 12